# Recommender Systems
This notebook contains the first part of assignment 1 about recommender systems.

By:
Antoni
Art Schenkel (3745244) (j.a.schenkel@umail.leidenuniv.nl)
Sadaf

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

get_ipython().run_line_magic('matplotlib', 'inline')


Reading the data, we will mainly use the rating_df for learning and testing 

In [13]:
movies_df = pd.read_csv('ml-1m/movies.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['Movie_Id', 'movie_name', 'genre'], encoding = "ISO-8859-1")

In [5]:
rating_df = pd.read_csv('ml-1m/ratings.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['user_id', 'Movie_Id','Ratings','Time_stamp'], encoding = "ISO-8859-1")

In [6]:
users_df = pd.read_csv('ml-1m/users.dat',
                        delimiter='::', header=None,
                        names=['user_id', 'Gender','Age','Occupation','Zip-Code'], encoding = "ISO-8859-1")

C:\Users\Art\AppData\Local\Temp\ipykernel_17188\3957394905.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users_df = pd.read_csv('ml-1m/users.dat',


## Cross-validation
The first part of the code implements the cross-validation. The cross-validation funtion return a dataframe with a randomly assigned number of folds. This dataframe is then used to learn and test the models, by choosing one of the fold as a valid test, and the other as the train sets.

In [28]:
def cross_validation(df,n_folds):
    shuffled_df = df.sample(random_state = 42, frac =1)
    shuffled_df['Fold']= None
    shuffled_df.reset_index(inplace = True)
    shuffled_df.drop(columns = 'index', inplace = True)
    data_size = len(shuffled_df)
    for i in range(1,n_folds):
        shuffled_df.loc[int((i-1)/n_folds*data_size):int(i/n_folds * data_size),'Fold'] = i
    shuffled_df.loc[int((n_folds-1)/n_folds*data_size):,'Fold']= n_folds
    return shuffled_df

data = cross_validation(rating_df, 5)
data.reset_index(inplace = True)

## Naive Approaches
We will now start with the naive approaches. These are: global average (ratingGlobal()), movie average (ratingItem), user average (ratingUser()) and a linear combination of the three averages(ratingUserItem()).

In [8]:
# rating global, return mean of all ratings
def ratingGlobal():
    return rating_df["Ratings"].mean()

print("Global rating:" , ratingGlobal())

Global rating: 3.581564453029317


In [14]:
# rating item, return mean of all ratings for a specific item
def ratingItem(item):
    join = pd.merge(movies_df, rating_df, how='left', on="Movie_Id")
    result = join[join["Movie_Id"] == item]
    return result["Ratings"].mean()

print("Item rating:", ratingItem(1193))

Item rating: 4.390724637681159


In [16]:
# rating user, return mean of all rating for a specific user
def ratingUser(user):
    join = pd.merge(users_df, rating_df, how='left', on="user_id")
    result = join[join["user_id"] == user]
    return result["Ratings"].mean()

print("User Rating:", ratingUser(3))

User Rating: 3.9019607843137254


In [18]:
# rating user item, combines user rating and item rating multiplied by paramer alpha and beta respectively. Lastly parameter gamma is added
def ratingUserItem(user, item, alpha, beta, gamma):
    return alpha * ratingUser(user) + beta * ratingItem(item) + gamma

print("User Item Rating:", ratingUserItem(1, 1193, 0.5, 0.5, 0.1))

User Item Rating: 4.389701941482089


In [20]:
# rating user star item is similar to rating user item, but wwithout adding parameter gamma at the end
def ratingUserStarItem(user, item, alpha, beta):
    return alpha * ratingUser(user) + beta * ratingItem(item)

print("User* Item Rating:", ratingUserStarItem(1, 1193, 0.5, 0.5))

User* Item Rating: 4.28970194148209


In [22]:
# This helper function returns a rating given a user_id and movie_id
def rating(user, item):
    result1 = rating_df[rating_df["user_id"] == user]
    result2 = result1[result1["Movie_Id"] == item]
    return int(result2.Ratings)

print(rating(1, 661))

3


C:\Users\Art\AppData\Local\Temp\ipykernel_17188\648485187.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(result2.Ratings)


In [23]:
# This function calculates the optimal value for parameters alpha, beta and gamma for a specific user and movie using linear regression.
def linearRegression(user, item):
    avguser = np.array([ratingUser(user)])
    avgmovie = np.array([ratingItem(item)])
    currrating = np.array([rating(user, item)])

    a = np.column_stack((avguser, avgmovie, np.ones_like(avguser)))
    b = currrating

    coefficients, residuals, rank, singular_values = np.linalg.lstsq(a, b, rcond=None)

    alpha, beta, gamma = coefficients

    print("Optimal values:")
    print("Alpha:", alpha)
    print("Beta:", beta)
    print("Gamma:", gamma)

linearRegression(1,661)

Optimal values:
Alpha: 0.4113321969726794
Beta: 0.34024284095705803
Gamma: 0.09820092990789195


C:\Users\Art\AppData\Local\Temp\ipykernel_17188\648485187.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(result2.Ratings)


In [ ]:
# apply linear regression to all data by filling avguser and avgmovie completely. also use fallback rule (global average)

## UV Matrix Decomposition
Next we implemented UV matrix decomposition as described in section 9.4 of the MMDS textbook.

In [26]:
def uvMatrixDecomp():
    DR = rating_df
    kf = KFold(n_splits = 5 , shuffle = True)
    c = 2
    i = 5
    for train_index , test_index in kf.split(DR):
        DR_train, DR_test = DR.loc[train_index], DR.loc[test_index]
        Row = DR_train.pivot(index = 'user_id', columns ='Movie_Id', values = 'Ratings')
        u_mean = Row.mean(axis=1)
        Row_array = Row.to_numpy()
        u_mean = u_mean.to_numpy()
        normal = Row_array - u_mean.reshape(-1,1)
        N = normal
        u = np.full((normal.shape[0],2), 1)
        v = np.full((2,normal.shape[1]), 1)
        u = u.astype(np.float32)
        v = v.astype(np.float32)
        uv = np.dot(u,v)
        print("TRAIN:", train_index, "TEST:", test_index)
        for iterations in range(i):
            for r in range(940):

                for s in range(c):
                    sums = 0
                    u_rk = u[r,:]
                    v_kj = v[:,:]
                    u_rk_del = np.delete(u_rk, s, 0)
                    v_kj_del = np.delete(v_kj, s, 0)
                    v_sj = v[s,:]
                    v_sj_squared = v_sj ** 2

                    u_rk_v_kj = np.dot(u_rk_del, v_kj_del)
                    m_rj = N[r,:]

                    error = m_rj - u_rk_v_kj

                    vsj_dot_er = v_sj * error
                    sums = np.nansum(vsj_dot_er)
                    v_sj_ssum = np.nansum((v_sj_squared) * (~np.isnan(m_rj)))
                    newval_u = sums / v_sj_ssum
                    u[r,s] = u[r,s] + ((newval_u - u[r,s]))
            for r in range(c):
                for s in range(Row_array.shape[1]):
                    sums = 0
                

                    u_ik = u[:,:]
                    v_ks = v[:,s]
                    u_ik_del = np.delete(u_ik, r, 1)

                    v_ks_del = np.delete(v_ks, r, 0)
                    u_ir = u[:,r]
                    u_ir_squared = u_ir ** 2

                    u_ik_v_ks = np.dot(u_ik_del, v_ks_del)
                    m_is = N[:,s]
                    error = m_is - u_ik_v_ks

                    uir_dot_er = u_ir * error
                    sumsv = np.nansum(uir_dot_er)
                    u_ir_ssum = np.nansum(u_ir_squared * (~np.isnan(m_is)))
                    newval_v = sumsv / u_ir_ssum
                    v[r,s] = v[r,s] + ((newval_v - v[r,s]))
            uv = np.dot(u,v)
            dif = uv -normal
            print("Iteration Number: ",iterations )
            dif_abs= (np.absolute(dif))
            dif_abs_0s = np.nan_to_num(dif_abs)
            dif_abs_sum = np.sum(dif_abs_0s,axis=0)
            sum_dif = dif_abs_sum.sum()
            non_0_count = np.count_nonzero(dif_abs_0s)
            MAE=sum_dif/non_0_count
            print('MAE',MAE)
            dif_sqr = dif ** 2
            dif_sqr_0s = np.nan_to_num(dif_sqr)
            dif_sqr_total= np.sum( dif_sqr_0s ,axis=0)
            sumz = dif_sqr_total.sum()
            non_0_count_sqr = np.count_nonzero( dif_sqr_0s )
            RME = sumz/ non_0_count_sqr
            rme_list=[RME]
            print('RMSE=',RME)

In [27]:
# perform the UV matrix decomposition
uvMatrixDecomp()

TRAIN: [      0       1       2 ... 1000206 1000207 1000208] TEST: [     16      19      22 ... 1000187 1000194 1000204]
Iteration Number:  0
MAE 0.8302802179206353
RMSE= 1.1312279668339145
Iteration Number:  1
MAE 0.7232146960318103
RMSE= 0.840447876967069
Iteration Number:  2
MAE 0.7205743706384972
RMSE= 0.8343832578566551
Iteration Number:  3
MAE 0.7201799563618158
RMSE= 0.8335733769097614
Iteration Number:  4
MAE 0.7200168663897022
RMSE= 0.8332424202635018
TRAIN: [      0       1       2 ... 1000205 1000207 1000208] TEST: [      4       6       7 ... 1000190 1000193 1000206]
Iteration Number:  0
MAE 0.8297374488366678
RMSE= 1.1300575859451367
Iteration Number:  1
MAE 0.7227668366681609
RMSE= 0.8388680146046142
Iteration Number:  2
MAE 0.7201789934358878
RMSE= 0.8327903835780063
Iteration Number:  3
MAE 0.7197928845117092
RMSE= 0.8319625916509858
Iteration Number:  4
MAE 0.7196270415512771
RMSE= 0.8316070915581323
TRAIN: [      0       1       2 ... 1000205 1000206 1000208] TEST: [ 

## Matrix Factorization
Lastly we implemented matrix factorization as described in the gravity-Tikk paper. Implementing the model, we create the model as two matrixes of features for the users and movies. Sizes of the matrixes are determined by the unique number of user_id (accordingly movie_id for the item matrix) and the number of features that is chosen when creating the model. 

As the train set may include gaps in the numbering of either user_id or movie_id we use dictionaries to translate the movie_id (or user_id) to matrix_id

For the learning part, we use the algorithm described on the slides from the lecture, we calculate the prediction as a scalar multiplication of the according vectors in matrixes of users and items. Then we calculate the error and update the vectors by learning_rate*(error*user_matrix_vector- lamb*self.item_matrix_vector). As we can see there are two parameters learning rate and lambda that we can tune.

We initialize the matrixes with random numbers using np.random.rand function

The output is put into [1:5] interval by simply setting every output < 1 into 1 and every output > 5 as 5, other outputs are not changed.

In the testing function, when the test example uses user_id that did not appear in the train set (in case of a movie_id not appearing in the train set proceed similarly) we take the sum of the vectors for the corresponding movie_id and scale the sums of features of all movie_ids into a [1:5] interval, and read the value of the movie_id that is asked in the testing example (after the scaling). 

The assumption is that the bigger the sum of features the bigger the chance is for the movie to get a high score. This assumption is not 100% true, as for example having feature < 0 with a corresponding <0 feature in the user_id vector creates a positive result.

In [29]:
class MatrixFactorization:
    def __init__(self,x, num_features):
        #initilaze two matrixes that then multiply by each other to give a matrix of ratings
        
        user_size = np.unique(x['user_id']).shape[0]
        item_size = np.unique(x['Movie_Id']).shape[0]

        values_user = np.unique(x['user_id'])
        self.dict_user = {values_user[i] : i for i in range(len(values_user))}

        values_item = np.unique(x['Movie_Id'])
        self.dict_item = {values_item[i] : i for i in range(len(values_item))}
        
        self.user_matrix = np.random.rand(user_size,num_features)
        self.item_matrix = np.random.rand(item_size,num_features)
        
    def fit(self,x, learning_rate = 0.005, lamb = 0.05, n_iter = 10):
        for it in range(n_iter):
            tmp = 0
            for i in range(len(x)):
                user = x.loc[i]['user_id']
                item = x.loc[i]['Movie_Id']

                user_idx = self.dict_user[user]
                item_idx = self.dict_item[item]
                
                #calculate the error
                error = x.loc[i]['Ratings'] - min(max(np.matmul(self.user_matrix[user_idx],self.item_matrix[item_idx]),1),5)
                # update values
                self.user_matrix[user_idx] = self.user_matrix[user_idx] + learning_rate*(error*self.item_matrix[item_idx] - lamb*self.user_matrix[user_idx])
                self.item_matrix[item_idx] = self.item_matrix[item_idx] + learning_rate*(error*self.user_matrix[user_idx] - lamb*self.item_matrix[item_idx])

                tmp += 1
                if tmp%50000 ==0:
                    print(f'currently done: {tmp/len(x)} % of the iteration {it}')

        
        print('current iteration ended: '+str(it))
    def test(self,x):
        predictions = []
        for i in range(len(x)):
            
            user = x.loc[i]['user_id']
            item = x.loc[i]['Movie_Id']

            try:
                user_idx = self.dict_user[user]
                item_idx = self.dict_item[item]
                pred = min(max(np.matmul(self.user_matrix[user_idx],self.item_matrix[item_idx]),1),5)
                predictions.append(pred)
                
            except: #If there is no user
                try:
                    item_idx = self.dict_item[item]
                    sum_item = np.sum(self.item_matrix[item_idx])
                    sums = np.sum(self.item_matrix, axis = 1)
                    pred =  (sum_item- np.min(sums)) / (np.max(sums) - np.min(sums)) * (4) + 1
                    predictions.append(pred)
                
                except: # If there is no movie
                    user_idx = self.dict_user[user]
                    sum_user= np.sum(self.user_matrix[item_idx])
                    sums = np.sum(self.user_matrix, axis = 1)
                    pred =  (sum_user - np.min(sums)) / (np.max(sums) - np.min(sums)) * (4) + 1
                    predictions.append(pred)
                #calculate the error
            
        labels = np.array(x['Ratings'])
        predictions = np.array(predictions)
        rmse =  mean_squared_error(labels,predictions, squared = False)
        mse = mean_absolute_error(labels,predictions)

        return rmse, mse

## Conclusion part one
This concludes the first part of our report about recommender systems. For the result and data visualisation of our matrix factorization method, see part two of the report.